# Métodos de primer orden

Los **métodos de primer orden** son métodos iterativos basados en derivadas de primer orden del objetivo. Dado un punto de inicio $\boldsymbol{\theta}_0$, la iteración $t$ consiste en hacer un paso de actualización (de $\boldsymbol{\theta}$):
$$\boldsymbol{\theta}_{t+1}=\boldsymbol{\theta}_t+\eta_t\boldsymbol{d}_t$$
donde $\eta_t$ es el **tamaño de paso (step size)** o **factor de aprendizaje (learning rate)**, y $\boldsymbol{d}_t$ es una **dirección de descenso** como el negativo del **gradiente**, dado por $\boldsymbol{g}_t=\mathbf{\nabla}\mathcal{L}(\boldsymbol{\theta})\rvert_{\boldsymbol{\theta}_t}$.
Los pasos de actualización se suceden hasta que el método alcanza un punto estacionario, esto es,
de gradiente nulo.

## Dirección de descenso

Decimos que $\boldsymbol{d}$ es una **dirección de descenso** si existe un $\eta_{\text{max}>0}$ tal que
$$\mathcal{L}(\boldsymbol{\theta}+\eta\boldsymbol{d})<\mathcal{L}(\boldsymbol{\theta})%
\qquad\text{para todo}\;\eta\in(0, \eta_{\text{max}})$$
El gradiente en $\boldsymbol{\theta}_t$, $\boldsymbol{g}_t=\mathbf{\nabla}\mathcal{L}(\boldsymbol{\theta})\rvert_{\boldsymbol{\theta}_t}=\mathbf{\nabla}\mathcal{L}(\boldsymbol{\theta}_t)=\boldsymbol{g}(\boldsymbol{\theta}_t)$ apunta en la dirección de máximo crecimiento de $\mathcal{L}$, mientras que su negativo lo hace en la de máximo decrecimiento. En general, $\boldsymbol{d}_t$ es dirección de descenso si $\boldsymbol{d}_t^t\boldsymbol{g}_t<0$, esto es, si $\boldsymbol{d}_t$ y $\boldsymbol{g}_t$ forman un ángulo mayor de $90$ grados (en el plano que los contiene). Este resultado incluye $\boldsymbol{d}_t=-\mathbf{B}_t\boldsymbol{g}_t$ para toda matriz $\mathbf{B}_t$ definida positiva.

**Descenso más pronunciado (steepest descent)** escoge el negativo del gradiente como dirección de descenso:
$$\boldsymbol{\theta}_{t+1}=\boldsymbol{\theta}_t-\eta_t\boldsymbol{g}_t$$

**Ejemplo:** $\mathcal{L}=\theta^2$, $\theta_0=10$, $\eta_t=0.2$, tolerancia $0.01$

In [2]:
import numpy as np

grad, theta, eta, tol = lambda t: 2*t, 10.0, 0.2, 0.01
while True:
	delta = -eta * grad(theta)
	if np.all(np.abs(delta) <= tol):
		break
	theta += delta
	print(round(theta, 3))

6.0
3.6
2.16
1.296
0.778
0.467
0.28
0.168
0.101
0.06
0.036
0.022
